# What affects methane chemical lifetime?

- methane
- VOCs
- NOx
- Ozone
- HCs
- N2O
- climate

Here I suggest an override of the alpha scaling factor for methane that is calculated from multiple species.

Ozone itself is a function of other precursors: we do not include ozone as a direct influence on methane lifetime, and restrict ourselves to directly emitted anthropogenic species.

Gill Thornhill published two papers on methane lifetime: one on the chemical adjustments to lifetime, and one on the climate adjustments. Both effects will be included. We will 

1. take AerChemMIP multi-model means from Gill's papers
2. run the lifetime relationship to individual AerChemMIP models in Gill's papers
3. find a least squares fit with reasonable sensitivies across the historical
4. run a Monte Carlo that perturbs the sensitivity of lifetime to each emitted species
  - include the perturbation to soil lifetime too

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
import scipy.optimize

from tqdm import tqdm
from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, FAIR
from fair21.defaults import species_config_from_default

## Temperature data

Use observations 1850-2020, then simulate an SSP3-7.0 climate with a linear warming rate to 4C in 2100.

In [ ]:
df_temp = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = np.zeros(751)
gmst[100:271] = df_temp['gmst'].values
gmst[271:351] = np.linspace(gmst[270], 4, 80)
gmst[351:] = 4

## Get emissions and concentrations 

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
input = {}
hfc_input = {}

In [ ]:
conc_species = ['CH4', 'N2O']
hfc_species = ['HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']

for species in conc_species:
    input[species] = df_conc.loc[
        (df_conc['Scenario']=='ssp370') & (df_conc['Variable'].str.endswith(species)) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()
    
for species in hfc_species:
    species_rcmip_name = species.replace("-", "")
    hfc_input[species] = df_conc.loc[
        (df_conc['Scenario']=='ssp370') & (df_conc['Variable'].str.endswith(species_rcmip_name)) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
emis_species = ['CO', 'VOC', 'NOx']
for species in emis_species:
    input[species] = df_emis.loc[
        (df_emis['Scenario']=='ssp370') & (df_emis['Variable'].str.endswith(species)) & 
        (df_emis['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
input['temp'] = gmst

In [ ]:
# species_ids = {
#     'ch4': SpeciesID('CH4', Category.CH4),
#     'co2': SpeciesID('CO2', Category.CO2, run_mode=RunMode.CONCENTRATION),
#     'n2o': SpeciesID('N2O', Category.N2O, run_mode=RunMode.CONCENTRATION),
#     'co': SpeciesID('CO', Category.SLCF_OZONE_PRECURSOR),
#     'voc': SpeciesID('VOC', Category.SLCF_OZONE_PRECURSOR),
#     'nox': SpeciesID('NOx', Category.SLCF_OZONE_PRECURSOR),
# }

In [ ]:
radiative_efficiency = pd.Series({
    'HFC-125': 0.23378,
    'HFC-134a': 0.16714,
    'HFC-143a': 0.168,
    'HFC-152a': 0.10174,
    'HFC-227ea': 0.27325,
    'HFC-23': 0.19111,
    'HFC-236fa': 0.25069,
    'HFC-245fa': 0.24498,
    'HFC-32': 0.11144,
    'HFC-365mfc': 0.22813,
    'HFC-4310mee': 0.35731
})

In [ ]:
# hfc_erf = {}
# hfc_sum = 0
# for species in ['HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
#     'HFC-365mfc', 'HFC-4310mee']:
#     hfc_erf[species] = (input[species][269] * radiative_efficiency[species]/1000)
#     hfc_sum = hfc_sum + hfc_erf[species]

In [ ]:
hfc134a_eq = 0
for species in hfc_species:
    hfc134a_eq = hfc134a_eq + (hfc_input[species] * radiative_efficiency[species])/(radiative_efficiency['HFC-134a'])

In [ ]:
pl.plot(hfc134a_eq)
pl.plot(hfc_input['HFC-134a'])

In [ ]:
for species in hfc_species:
    pl.plot(hfc_input[species])

In [ ]:
for species in hfc_species:
    print(species, (hfc_input[species][264] * radiative_efficiency[species])/radiative_efficiency['HFC-134a']/hfc134a_eq[264], hfc_input[species][264]-hfc_input[species][100])

In [ ]:
hfc134a_eq[269]  # 237 ppm in SPM WG1 AR6

In [ ]:
input['HFC'] = hfc134a_eq

Use 1850 and 2014 emissions or concentrations corresponding to methane lifetime changes in Thornhill et al. 2021.

Could we also take into account the fact that there are multiple loss pathways for CH4:
- tropospheric OH loss is 560 Tg/yr
- chlorine oxidation, 11 Tg/yr, assumed not included in AerChemMIP models
- stratospheric loss is 31 Tg/yr, assumed not included in AerChemMIP models
- soil uptake, 30 Tg/yr, not included in AerChemMIP models

Saunois (2020): 90% of sink is OH chemistry in troposphere and is 553 [476–677] Tg CH4 yr−1, which is close to the IPCC number of 560, (chapter 5)

Chapter 6 only give time constants for soil uptake and the combined chemistry loss (trop OH + chlorine + stratosphere). 

In [ ]:
def alpha_scaling_exp(
    input,
    baseline,
    normalisation,
    beta,
):
    log_lifetime_scaling = 0
    for species in ['CH4', 'N2O', 'VOC', 'HFC', 'NOx', 'temp']:
        log_lifetime_scaling = log_lifetime_scaling + (
            np.log(1 + (input[species]-baseline[species])/normalisation[species] * beta[species])
        )
#    for species in ['HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea',
#                    'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc', 'HFC-4310mee']:
#        log_lifetime_scaling = log_lifetime_scaling + (
#            np.log(1 + 
#                (
#                    (input[species]-baseline[species])/(normalisation[species]) * beta['HC'] * hfc_erf[species]/hfc_sum
#                )
#            )
#        )
    return np.exp(log_lifetime_scaling)

In [ ]:
normalisation = {}
for species in ['CH4', 'N2O', 'VOC', 'NOx', 'HFC']:
    normalisation[species] = input[species][264] - input[species][100]
    print(species, normalisation[species])
normalisation['temp'] = 1

In [ ]:
baseline = {}
for species in ['CH4', 'N2O', 'VOC', 'NOx', 'HFC']:
    baseline[species] = input[species][100]
baseline['temp'] = 0

## Steps 1 and 2

Get and tune to AerChemMIP models

MRI and GISS both give pretty good historical emulations

In [ ]:
parameters = {}

parameters['AerChemMIP_mean'] = {
    'base': 10.0,
    'CH4': +0.22,
    'NOx': -0.33,
#    'CO': 0,
    'VOC': +0.19,
    'HFC': -0.037,
    'N2O': -0.02,
    'temp': 0.012,
}

parameters['UKESM'] = {
    'base': 8,
    'CH4': +0.22,
    'NOx': -0.25,
#    'CO': 0,
    'VOC': +0.11,
    'HFC': -0.049,
    'N2O': -0.012,
    'temp': 0.0043
}

# we'll exclude BCC and CESM as they don't have VOC expt and that's important. 
# We can live with a missing N2O from GFDL and a missing temperature feedback from MRI.

parameters['GFDL'] = {
    'base': 9.6,
    'CH4': +0.21,
    'NOx': -0.33,
#    'CO': 0,
    'VOC': +0.15,
    'HFC': -0.075,
    'N2O': 0,  # missing
    'temp': 0.0258
}

parameters['GISS'] = {
    'base': 13.4,
    'CH4': +0.18,
    'NOx': -0.46,
#    'CO': 0,
    'VOC': +0.27,
    'HFC': -0.006,
    'N2O': -0.039,
    'temp': -0.0167
}

parameters['MRI'] = {
    'base': 10.1,
    'CH4': +0.22,
    'NOx': -0.26,
#    'CO': 0,
    'VOC': +0.21,
    'HFC': -0.024,
    'N2O': -0.013,
    'temp': 0  # missing
}

In [ ]:
lifetime_scaling = {}

In [ ]:
models = ['AerChemMIP_mean', 'UKESM', 'GFDL', 'GISS', 'MRI']

In [ ]:
for model in models:
    print(parameters[model])
    lifetime_scaling[model] = alpha_scaling_exp(
        input,
        baseline,
        normalisation,
        parameters[model],
    )

In [ ]:
#pl.plot(np.arange(1750, 2501), aerchemmip_mean[:] * 8.25)
for model in models:
    pl.plot(np.arange(1750, 2501), lifetime_scaling[model] * parameters[model]['base'], label=model)
pl.legend()
pl.ylabel('CH4 chemical lifetime (yr)')

In [ ]:
#1/(1/135 + 1/9.7)

In [ ]:
#1/(1/120 + 1/200 + 1/150 + 1/11.2)

In [ ]:
1/np.inf

In [ ]:
# put this into a simple one box model
def one_box(
    emissions,
    gas_boxes_old,
    airborne_emissions_old,
    burden_per_emission,
    lifetime,
    alpha_lifetime,
    partition_fraction,
    pre_industrial_concentration,
    soil_lifetime=135,
    timestep=1,
    natural_emissions_adjustment=0,
):
    
    effective_lifetime = 1/(1/(alpha_lifetime * lifetime) + 1/soil_lifetime)
    decay_rate = timestep/(effective_lifetime)
    decay_factor = np.exp(-decay_rate)
    gas_boxes_new = (
        partition_fraction *
        (emissions-natural_emissions_adjustment) *
        1 / decay_rate *
        (1 - decay_factor) * timestep + gas_boxes_old * decay_factor
    )
    airborne_emissions_new = gas_boxes_new
    concentration_out = (
        pre_industrial_concentration +
        burden_per_emission * (
            airborne_emissions_new + airborne_emissions_old
        ) / 2
    )
    return concentration_out, gas_boxes_new, airborne_emissions_new

In [ ]:
emis_ch4 = df_emis.loc[
    (df_emis['Scenario']=='ssp370') & (df_emis['Variable'].str.endswith('CH4')) & 
    (df_emis['Region']=='World'), '1750':'2500'
].interpolate(axis=1).values.squeeze()

In [ ]:
burden_per_emission = 1 / (5.1352e18 / 1e18 * 16.043 / 28.97)
partition_fraction = 1
pre_industrial_concentration = 729.2
natural_emissions_adjustment = emis_ch4[0]

In [ ]:
conc_ch4 = {}

In [ ]:
for model in models:
    conc_ch4[model] = np.zeros(751)
    gas_boxes = 0
    airborne_emissions = 0
    for i in range(751):
        conc_ch4[model][i], gas_boxes, airborne_emissions = one_box(
            emis_ch4[i],
            gas_boxes,
            airborne_emissions,
            burden_per_emission,
            parameters[model]['base'],
            lifetime_scaling[model][i],
            partition_fraction,
            pre_industrial_concentration,
            soil_lifetime=135,
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment,
        )

In [ ]:
for model in models:
    pl.plot(np.arange(1750, 2501), conc_ch4[model], label=model)
pl.plot(np.arange(1750, 2501), input['CH4'], color='k', label='obs')
pl.legend()

In [ ]:
for model in models:
    pl.plot(np.arange(1750, 2021), conc_ch4[model][:271], label=model)
pl.plot(np.arange(1750, 2021), input['CH4'][:271], color='k', label='obs')
pl.legend()

## Step 3

Find least squares sensible historical fit

In [ ]:
invect = np.array([input['CH4'], input['NOx'], input['VOC'], input['HFC'], input['N2O'], input['temp']])

In [ ]:
def fit_precursors(x, rch4, rnox, rvoc, rhfc, rn2o, rtemp, rsoil, rbase):
    conc_ch4 = np.zeros(751)
    gas_boxes = 0
    airborne_emissions = 0
    
    params = {}
    params['CH4'] = rch4
    params['NOx'] = rnox
    params['VOC'] = rvoc
    params['HFC'] = rhfc
    params['N2O'] = rn2o
    params['temp'] = rtemp
    params['soil'] = rsoil
    
    inp = {}
    inp['CH4'] = x[0]
    inp['NOx'] = x[1]
    inp['VOC'] = x[2]
    inp['HFC'] = x[3]
    inp['N2O'] = x[4]
    inp['temp'] = x[5]
    
    lifetime_scaling = alpha_scaling_exp(
        inp,
        baseline,
        normalisation,
        params,
    )
    
    for i in range(751):
        conc_ch4[i], gas_boxes, airborne_emissions = one_box(
            emis_ch4[i],
            gas_boxes,
            airborne_emissions,
            burden_per_emission,
            rbase,
            lifetime_scaling[i],
            partition_fraction,
            pre_industrial_concentration,
            soil_lifetime=rsoil,
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment,
        )
    return conc_ch4


p, cov = scipy.optimize.curve_fit(
    fit_precursors, 
    invect[:, :751],
    input['CH4'],
    bounds = (  # AerChemMIP range +/- a little bit
        (0.18, -0.46, 0.11, -0.075, -0.039, -0.0167, 85, 6.3),
        (0.26, -0.25, 0.27, -0.006, -0.012, +0.0258, 185, 13.4)
    )
)

In [ ]:
parameters['best_fit'] = {
    'base': p[7],
    'CH4': p[0],
    'NOx': p[1],
#    'CO': 0,
    'VOC': p[2],
    'HFC': p[3],
    'N2O': p[4],
    'temp': p[5],
    'soil': p[6]
}
p

In [ ]:
for species in hfc_species:
    print(species, p[3] * (hfc_input[species][264] * radiative_efficiency[species])/radiative_efficiency['HFC-134a']/hfc134a_eq[264])

In [ ]:
lifetime_scaling['best_fit'] = alpha_scaling_exp(
    input,
    baseline,
    normalisation,
    parameters['best_fit'],
)

In [ ]:
pl.plot(np.arange(1750, 2501), lifetime_scaling['best_fit'] * parameters['best_fit']['base'], label='best_fit')
pl.legend()
pl.ylabel('CH4 chemical lifetime (yr)')

In [ ]:
conc_ch4['best_fit'] = np.zeros(751)
gas_boxes = 0
airborne_emissions = 0
for i in range(751):
    conc_ch4['best_fit'][i], gas_boxes, airborne_emissions = one_box(
        emis_ch4[i],
        gas_boxes,
        airborne_emissions,
        burden_per_emission,
        parameters['best_fit']['base'],
        lifetime_scaling['best_fit'][i],
        partition_fraction,
        pre_industrial_concentration,
        soil_lifetime=parameters['best_fit']['soil'],
        timestep=1,
        natural_emissions_adjustment=natural_emissions_adjustment,
    )

In [ ]:
pl.plot(np.arange(1750, 2501), conc_ch4['best_fit'], label='best_fit')
pl.plot(np.arange(1750, 2501), input['CH4'], color='k', label='obs')
pl.legend()

### Compare the SSP3-7.0 fit to other SSPs

should do something with the temperature projections here

In [ ]:
emis_ch4_ssps = {}

for ssp in ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']:
    emis_ch4_ssps[ssp] = df_emis.loc[
        (df_emis['Scenario']==ssp) & (df_emis['Variable'].str.endswith('CH4')) & 
        (df_emis['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
for ssp in ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']:
    conc_ch4[ssp] = np.zeros(751)
    gas_boxes = 0
    airborne_emissions = 0
    for i in range(751):
        conc_ch4[ssp][i], gas_boxes, airborne_emissions = one_box(
            emis_ch4_ssps[ssp][i],
            gas_boxes,
            airborne_emissions,
            burden_per_emission,
            parameters['best_fit']['base'],
            lifetime_scaling['best_fit'][i],
            partition_fraction,
            pre_industrial_concentration,
            soil_lifetime=parameters['best_fit']['soil'],
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment,
        )

In [ ]:
for ssp in ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']:
    pl.plot(np.arange(1750, 2501), conc_ch4[ssp], label=ssp)
#pl.plot(np.arange(1750, 2501), input['CH4'], color='k', label='obs')
pl.legend()

## Step 4

Run a Monte Carlo around these parameters

In [ ]:
samples = 25000

In [ ]:
param_mc = []
base = scipy.stats.uniform.rvs(9, 15-9, size=samples, random_state=1001251)
#ch4 = scipy.stats.uniform.rvs(0.18, 0.26-0.18, size=samples, random_state=21260)  # extend?
ch4 = scipy.stats.uniform.rvs(0.12, 0.32-0.12, size=samples, random_state=21260)
#nox = scipy.stats.uniform.rvs(-0.46, -0.25-(-0.46), size=samples, random_state=3936801)
nox = scipy.stats.uniform.rvs(-0.53, -0.18-(-0.53), size=samples, random_state=3936801)
co = np.zeros(samples)
#voc = scipy.stats.uniform.rvs(0.11, 0.27-0.11, size=samples, random_state=10372947)
voc = scipy.stats.uniform.rvs(0.05, 0.33-0.05, size=samples, random_state=10372947)
hfc = scipy.stats.uniform.rvs(-0.075, -0.006-(-0.075), size=samples, random_state=9165539)
n2o = scipy.stats.uniform.rvs(-0.039, -0.012-(-0.039), size=samples, random_state=442935)
#temp = scipy.stats.uniform.rvs(-0.0167, 0.0258-(-0.0167), size=samples, random_state=1930159)
temp = scipy.stats.uniform.rvs(-0.025, 0.035-(-0.025), size=samples, random_state=1930159)
soil = scipy.stats.uniform.rvs(135-44*2, 135+44*2, size=samples, random_state=6645293)

for sample in range(samples):
    param_mc.append(
        {
            'base': base[sample],
            'CH4': ch4[sample],
            'NOx': nox[sample],
            'CO': co[sample],
            'VOC': voc[sample],
            'HFC': hfc[sample],
            'N2O': n2o[sample],
            'temp': temp[sample],
            'soil': soil[sample]
        }
    )

In [ ]:
conc_ch4_mc = {}
for sample in tqdm(range(samples)):
    conc_ch4_mc[sample] = np.zeros(751)
    gas_boxes = 0
    airborne_emissions = 0
    lifetime_scaling_mc = alpha_scaling_exp(
        input,
        baseline,
        normalisation,
        param_mc[sample],
    )
    for i in range(751):
        conc_ch4_mc[sample][i], gas_boxes, airborne_emissions = one_box(
            emis_ch4[i],
            gas_boxes,
            airborne_emissions,
            burden_per_emission,
            param_mc[sample]['base'],
            lifetime_scaling_mc[i],
            partition_fraction,
            pre_industrial_concentration,
            soil_lifetime=param_mc[sample]['soil'],
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment,
        )

In [ ]:
input['CH4'][264]

In [ ]:
accept = np.zeros(samples, dtype=bool)
for sample in range(samples):
    if input['CH4'][264] - 10 < conc_ch4_mc[sample][264] < input['CH4'][264] + 10:
        accept[sample] = True

In [ ]:
np.sum(accept)

In [ ]:
for sample in range(samples):
    if accept[sample]:
        pl.plot(np.arange(1750, 2501), conc_ch4_mc[sample])
pl.plot(np.arange(1750, 2501), input['CH4'], color='k')

In [ ]:
for sample in range(samples):
    if accept[sample]:
        pl.plot(np.arange(1750, 2021), conc_ch4_mc[sample][:271])
pl.plot(np.arange(1750, 2021), input['CH4'][:271], color='k')

In [ ]:
pl.hist(base[accept])  # we're wasting time looking below tau = 9, and we can explore more tau > 13

In [ ]:
pl.hist(ch4[accept])  # no surprise that CH4 sensitivity is probably overconstrained

In [ ]:
pl.hist(nox[accept])  # a wide range of nox sensitivities are possible

In [ ]:
pl.hist(voc[accept])

In [ ]:
pl.hist(temp[accept])

In [ ]:
pl.hist(soil[accept])

In [ ]:
1 / (1/10 + 1/180)

In [ ]:
(1/12) / (1/10 + 1/180)

In [ ]:
(1) / ((1/12)/10 + (1/12)/180)

In [ ]:
# decay rate yearly
1 / 10

In [ ]:
# decay rate monthly
(1/12) / 10

In [ ]:
# new decay rate yearly
1/ (1 / (1/10 + 1/180))

In [ ]:
# new decay rate monthly
(1/12) / (1 / (1/10 + 1/180))

In [ ]:
(1) / (1/12 / (1/10 + 1/180))